### 1-1번 기본 체인 만들기 -AI 요리사

In [1]:
print('Hello LangChain')

Hello LangChain


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#print(OPENAI_API_KEY)

In [10]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 요리사 입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="계란과 밥 김치로 만들 수 있는 요리 단계별로 설명과 음식을 추천해줘")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 요리사 입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [11]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000250C93C7890> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000250C93C7C50> root_client=<openai.OpenAI object at 0x00000250C93C5E50> root_async_client=<openai.AsyncOpenAI object at 0x00000250C93C7390> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [13]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: 계란과 밥, 김치로 만들 수 있는 요리로 '계란 김치밥'을 추천해 드리겠습니다.

1.  **재료 준비**: 계란 2개, 밥 1공기, 김치 1/2컵, 참기름 1티스푼, 소금 1/2티스푼, 후추 1/4티스푼을 준비합니다.
2.  **김치 다듬기**: 김치를 잘게 썰어주세요. 
3.  **계란 풀기**: 계란을 깨뜨려 그릇에 넣고, 소금과 후추로 간을 한 뒤 잘 풀어줍니다.
4.  **계란 프라이**: 팬에 참기름을 두르고 계란을 넣어 스크램블 에그처럼 만들어줍니다. 
5.  **밥과 김치 섞기**: 밥에 썰어 놓은 김치를 넣고 잘 섞어줍니다.
6.  **완성**: 이제 프라이팬에 만들어 놓은 계란과 김치밥을 섞어줍니다. 
7.  **마무리**: 맛있는 계란 김치밥이 완성되었습니다!


### 1-2 2단계 체인 만들기 - 영화 추천 시스템
* 문제 설명 : 사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그영화의 정보(감독, 줄거리, 등장인물)들을 알려주는 2단계 체인을 구현해보세요.

In [45]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#print(OPENAI_API_KEY)

In [46]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 영화감평사 입니다. 장르에 맞는 최고의 영화를 추천하고 이유를 설명해주세요.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 영화감평사 입니다. 장르에 맞는 최고의 영화를 추천하고 이유를 설명해주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

#
prompt3 = ChatPromptTemplate.from_template("{movie}  감독이 누구인지 알려주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

client=<openai.resources.chat.completions.completions.Completions object at 0x00000250C96CB020> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000250C96CA8A0> root_client=<openai.OpenAI object at 0x00000250C96CB110> root_async_client=<openai.AsyncOpenAI object at 0x00000250C96CAD50> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [54]:
# 체인 정의
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()
chain3 = prompt3 | llm | StrOutputParser()

# 실행
output1 = chain1.invoke({"genre": "스릴러"})      # 장르로 영화 추천
output2 = chain2.invoke({"movie": output1})       # 추천 영화로 줄거리 요약
output3 = chain3.invoke({"movie": output1})       # 추천 영화의 감독 정보

print("🎬 추천 영화:", output1)
print("📝 줄거리 요약:\n", output2)
print("🎥 감독:", output3)

🎬 추천 영화: 스릴러 영화 중 '올드 보이'(2003)를 추천합니다. 

올드 보이는 한국 영화계에서 가장 영향력 있는 영화 중 하나입니다. 이 영화는 주인공이 15년간 감금된 채로 기억을 잃고, 그 후 기억을 되찾고 복수를 준비하는 내용을 담고 있습니다. 

이 영화는 심리 스릴러 장르에 속하며, 긴장감 넘치는 스토리와 뛰어난 연기력으로 많은 팬을 보유하고 있습니다.
📝 줄거리 요약:
 추천하는 영화는 '올드 보이'(2003)입니다.

주인공은 15년간 감금된 채로 기억을 잃고, 감금된 이유나 자신의 과거에 대해 전혀 모르는 상태입니다. 그러나 기억을 되찾은 후, 그는 자신의 감금과 복수를 준비하게 됩니다. 이 영화는 긴장감 넘치는 스토리와 뛰어난 연기력으로 많은 팬을 보유하고 있습니다.
🎥 감독: 올드 보이의 감독은 박찬욱입니다.


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 

In [6]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI

# 1. 예시 프롬프트: 변수명을 examples에 맞춰서 news, keywords로 변경
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 2. 예시 데이터
examples = [
    {
        "news": "삼성전자가 차세대 고성능 HBM4 메모리 개발에 착수했다. 이번 제품은 AI 서버와 데이터 센터에서 성능 향상에 기여할 것으로 기대된다.",
        "keywords": "삼성전자, HBM4, AI서버"
    },
    {
        "news": "한국은행이 기준금리를 동결하면서 올해 하반기 경기 회복에 대한 기대가 커지고 있다. 소비자물가는 여전히 높은 수준을 유지하고 있다.",
        "keywords": "한국은행, 기준금리, 소비자물가"
    },
    {
        "news": "정부는 이번 국무회의에서 청년 일자리 창출과 관련된 2025년 정책 로드맵을 발표했다. 디지털 산업과 친환경 에너지 분야가 핵심으로 포함되었다.",
        "keywords": "정부, 청년일자리, 친환경에너지"
    },
    {
        "news": "전 세계적으로 이산화탄소 배출량이 증가하고 있으며, 유엔은 각국에 기후변화 대응 강화를 촉구했다. 특히 선진국의 책임이 강조되고 있다.",
        "keywords": "이산화탄소, 기후변화, 유엔"
    }
]
# 3. Few-Shot Prompt 구성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

# 5. LLM 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)

# 6. 체인 생성
chain = final_prompt | llm

# 7. 테스트 뉴스 실행
# test_news = """한국은행은 6월 금융통화위원회 회의에서 기준금리를 3.50%로 유지하기로 결정했습니다.  
# 이번 결정은 물가 상승률 둔화와 경기 불확실성 사이에서 균형을 고려한 결과로 해석됩니다.  
# 이창용 총재는 향후 경제 지표와 대외 여건에 따라 유연하게 대응할 것이라고 밝혔습니다."""

# test_news = """"제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
# 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
# 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다." """

test_news = """[이데일리 윤정훈 기자] 국산 인공지능(AI) 인프라 전문기업 모레(Moreh)의 자회사 모티프테크놀로지가 그래픽처리장치(GPU) 1개로 추론이 가능한 소형언어모델(sLLM)을 최초로 공개했다. 저전력으로 구동되고 슈퍼컴퓨터 없이 운영이 가능한 장점을 바탕으로 국내외 AI 생태계 공략에 나선다는 계획이다.


임정환 모티프테크놀로지스 대표가 10일 서울 강남구 조선팰리스에서 열린 ‘레노보 테크데이’에서 sLLM 모델 ‘모티프 2.6B’를 소개하고 있다.(사진=윤정훈 기자)
모티프테크놀로지는 10일 레노보 테크데이에 참석해 프롬 스크래치(from scratch·밑바닥부터) 개발한 파운데이션 sLLM ‘모티프 2.6B’를 오픈소스로 허깅페이스에 공개했다고 밝혔다. 이 모델은 AMD가 개발한 인스텔라를 제외하고 AMD 인스팅트 MI250 GPU 기반으로 구현한 최초의 AI 파운데이션 모델이다.

임정환 모티프테크놀로지스 대표는 “sLLM은 저전력으로 구동되고 슈퍼컴퓨터 없이 운영이 가능해 비용 효율성이 매우 높아 실제 산업 현장에서 다양한 적용이 가능해 성장 잠재력이 매우 크다”면서 “이번에 선보인 모티프2.6B를 활용해 우리 일상에서 사용할 수 있는 온디바이스 AI, 에이전틱 AI 모델로 발전시켜 나갈 것”이라고 강조했다.

이번에 모티프가 공개한 sLLM은 모회사인 모레가 설립 초기부터 추구해온 GPU 자원의 효율적 사용과 클러스터링 SW 최적화 기술을 기반으로 모티프에서 개발한 경량화된 고성능 AI모델이다.

모레는 작년 12월 오픈AI GPT-4의 한국어 성능을 능가하는 1020억 매개변수 규모의 한국어 특화 고성능 LLM을 개발했고, 올해 2월부터는 법인을 독립해 AMD GPU 기반의 AI모델 개발에 힘써왔다.

모티프는 26억개 매개변수로 구성된 모티프 2.6B가 글로벌 sLLM과 비교해도 성능이 뛰어나다고 밝혔다.

각 개발사가 공개한 테크니컬 리포트의 점수와 설정값을 동일하게 적용해 벤치마크 점수를 산출한 결과 ‘모티프 2.6B’는 70억 개 매개변수를 가진 미스트랄 7B 대비 134%의 성능을 보였다. 특히 고성능을 요하는 고난도 수학 및 과학, 코딩 능력에서 상대적으로 높은 점수를 기록했다. 동급인 1B~3B 모델과의 비교에서도 구글 젬마1(2B) 대비 191%, 메타 라마 3.2(1B) 대비 139%, AMD 인스텔라(3B) 대비 112%, 알리바바 큐원 2.5(3B) 104%로 우수한 성능을 보였다.

‘모티프 2.6B’는 문맥 이해 능력을 강화한 점이 가장 큰 기술적 특징이다. 잘못된 문맥을 참고해 부정확한 문장을 생성하는 오류를 줄이고, 필수적인 핵심 문맥에 집중하도록 설계했다. 트랜스포머(Transformer) 구조의 핵심인 어텐션(Attention) 기술을 보다 정교하게 활용해 좀 더 적절하게 단어를 사용할 수 있는 구조를 적용했다.


모티프가 만든 sLLM 모델을 구글, MS, 알리바바 등의 동급 이상의 모델과 성능을 비교한 표(사진=모티프테크놀로지)
모레는 모티프가 공개한 sLLM으로 국내 AX 시장 진출을 하는 동시에 레노버·AMD와 파트너십을 맺고 AI 데이터센터 솔루션 시장 진출도 추진하고 있다. 현재 중국, 인도, 일본 등 시장에서 10여 곳의 고객사가 도입을 검토 중이다.

조형근 모레 최고전략책임자(CSO)는 “모레는 엔비디아 의존 없이 AMD와 협력해 효율적인 AI인프라를 만들어서 검증을 마쳤다”며 “많은 기업이 저희의 인프라 SW와 기술을 활용해 고효율의 경제성 있는 AI를 만들어 주길 바란다”고 말했다.

"""

# 8. 실행
result = chain.invoke({"input": test_news})
print(result.content)


키워드: 모티프테크놀로지, sLLM, AI모델


# 문제 2-1 : 콤마 구분 리스트 파서 활용

In [31]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()


In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
import csv
import os

# 디렉터리 생성
os.makedirs("./data", exist_ok=True)

# 출력 포맷 지침 (한국어 요청)
format_instructions = "결과는 쉼표(,)로 구분된 한국어 목록으로 제공해 주세요."

# 프롬프트 템플릿
prompt = PromptTemplate(
    template="다섯 가지 {subject}를 알려주세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 체인 연결
chain = prompt | model

# 프롬프트 주제 한국어로 설정
result = chain.invoke({"subject": "사용자가 관심있는 분야인 음식,스포츠,영화"})

# 응답 추출
output_text = result.content if hasattr(result, "content") else str(result)

# 출력 확인
print("한국의 유명한 장소/활동/음식 목록:")
print(output_text)

# CSV 저장
csv_filename = "./data/연습문제 2-1.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["장소/활동/음식"])
    for item in output_text.split(","):
        writer.writerow([item.strip()])

print(f"'{csv_filename}' 파일로 저장 완료!")


한국의 유명한 장소/활동/음식 목록:
사용자가 관심있는 분야를 5가지로 제한해서 음식, 스포츠, 영화를 알려드릴게요.
결과는 쉼표로 구분된 한국어 목록으로 제공해 드릴게요: 음식, 스포츠, 영화, 음악, 여행
'./data/연습문제 2-1.csv' 파일로 저장 완료!


In [43]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
import csv
import os
import ast
import re
import httpx  # httpx 예외 처리를 위해 import

os.makedirs("./data", exist_ok=True)

format_instructions = "결과는 Python 리스트 형식으로 반환해 주세요. 예: ['항목1', '항목2', '항목3', '항목4', '항목5']"

prompt = PromptTemplate(
    template="한국에서 유명한 {subject} 다섯 가지를 알려주세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

subject = "음식"

chain = prompt | model

try:
    result = chain.invoke({"subject": subject})
    output_text = result.content if hasattr(result, "content") else str(result)
except httpx.RequestError as e:
    print(f"⚠️ 네트워크 오류 발생: {e}")
    output_text = "[]"  # 빈 리스트 문자열로 기본값 설정
except Exception as e:
    print(f"⚠️ 예상치 못한 오류 발생: {e}")
    output_text = "[]"

# 리스트 부분 추출 및 파싱
match = re.search(r"\[.*\]", output_text, re.DOTALL)
if match:
    list_str = match.group(0)
    try:
        items = ast.literal_eval(list_str)
        items = [item for item in items if '.' not in item and 'python' not in item.lower()]
    except Exception as e:
        print("⚠️ 리스트 파싱 실패:", e)
        items = [output_text]
else:
    print("⚠️ 리스트 형식이 감지되지 않음.")
    items = [output_text]

print(f"한국의 유명한 {subject} 목록:")
print(items)

csv_filename = f"./data/korea_{subject}.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([subject])
    for item in items:
        writer.writerow([item.strip()])

print(subject)


한국의 유명한 음식 목록:
['비빔밥', '불고기', '김치찌개', '삼계탕', '냉면']
음식


In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
import csv
import os
import ast
import re
import httpx

os.makedirs("./data", exist_ok=True)

format_instructions = "결과는 Python 리스트 형식으로 반환해 주세요. 예: ['항목1', '항목2', '항목3', '항목4', '항목5']"

prompt = PromptTemplate(
    template="한국에서 유명한 {subject} 다섯 가지를 알려주세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

subject = "기업"

chain = prompt | model

try:
    result = chain.invoke({"subject": subject})
    output_text = result.content if hasattr(result, "content") else str(result)
except httpx.RequestError as e:
    print(f"⚠️ 네트워크 오류 발생: {e}")
    output_text = "[]"
except Exception as e:
    print(f"⚠️ 예상치 못한 오류 발생: {e}")
    output_text = "[]"

# 응답에서 리스트 부분만 추출
# 1) ```python ... ``` 코드 블록 내 리스트 추출
code_block_match = re.search(r"```python\s*\n(.+?)\n```", output_text, re.DOTALL)
if code_block_match:
    code_block_content = code_block_match.group(1)
    # 리스트를 포함하는 줄만 추출 (보통 famous_cars = [...] 같은 형식)
    list_match = re.search(r"\[.*\]", code_block_content, re.DOTALL)
    if list_match:
        list_str = list_match.group(0)
    else:
        list_str = ""
else:
    # 코드블록 없으면 그냥 텍스트에서 리스트 찾기 시도
    list_match = re.search(r"\[.*\]", output_text, re.DOTALL)
    list_str = list_match.group(0) if list_match else ""

# 리스트 문자열이 있으면 파싱 시도
if list_str:
    try:
        items = ast.literal_eval(list_str)
    except Exception as e:
        print("⚠️ 리스트 파싱 실패:", e)
        items = [output_text]
else:
    print("⚠️ 리스트 형식이 감지되지 않음.")
    items = [output_text]

print(f"한국의 유명한 {subject} 다섯 가지 목록입니다.")
print(items)

csv_filename = f"./data/korea_{subject}.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([subject])
    for item in items:
        writer.writerow([item.strip()])

print(f"'{csv_filename}' 파일로 저장 완료!")

한국의 유명한 기업 다섯 가지 목록입니다.
['삼성전자', 'LG전자', '현대자동차', '네이버', '카카오']
'./data/korea_기업.csv' 파일로 저장 완료!
